In [1]:
#Importations utiles liés à la gestion des données
import pandas as pd
from scipy import sparse
from scipy import linalg
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import tqdm.notebook as tq
import pickle

In [2]:
def fp_from_fp_db(fp_db):
    '''
    Cette fonction permet de recréer le fingerprint de taille 2048
    à partir du fingerprint codé en mémoire
    '''
    res = np.zeros(2048)
    #On initialise un array de 2048 zéros
    list_idx = fp_db.split(" ")
    #On récupère tous les indices non nuls du fingerprint
    for el in list_idx[:-1]:
        lfp = el.split("-")
        n = len(lfp)
        #En fonction des cas, on complète avec la bonne valeur
        if n == 1:
            res[int(lfp[0])] = 1
        elif n==2:
            res[int(lfp[0])] = int(lfp[-1])
        else:
            res[int(lfp[0])] = -int(lfp[-1])
    return res

def input_model_from_db(base_fp):
    '''
    Cette fonction renvoie un array de fingerprint à partir d'un dataset contenant des
    fingerprints sous forme "codée"
    '''
    #On sépare fingerprint de réactions et fingerprints du produit
    prod_fp_db,rxn_fp_db = base_fp["Product_Fingerprint"],base_fp["Reaction_Fingerprint"]
    prod_fp,rxn_fp = [],[]
    p_bar = tq.tqdm(total=base_fp.shape[0])
    for i in range(base_fp.shape[0]):
        #On essaie de récupérer le fingerprint associé à la ligne i
        try:
            p_bar.update(1)
            rxn_fp.append(fp_from_fp_db(rxn_fp_db.iloc[i]))
            prod_fp.append(fp_from_fp_db(prod_fp_db.iloc[i]))
        except:
            continue
    return np.array(prod_fp), np.array(rxn_fp)

In [3]:
#Nom des fichiers nécessaires
fname_fp_false = "NOM DU FICHIER CONTENANT LES FAUSSES REACTIONS SUR LESQUELLES S'ENTRAINER"
fname_fp_true = "NOM DU FICHIER CONTENANT LES VRAIES  REACTIONS SUR LESQUELLES S'ENTRAINER"

In [4]:
#On récupère les bases de données associées aux noms des fichiers
data_fp_false = pd.read_csv(fname_fp_false)
data_fp_true = pd.read_csv(fname_fp_true)

In [5]:
#On récupère la taille des données
size_true = data_fp_true.shape[0]
size_false = data_fp_false.shape[0]

In [6]:
#On convertit les données en fingerprint 
#On tronque pour ne pas saturer la RAM
size_percentage = 0.1
#On récupère les produits et les réactions correspondantes sous forme de fingerprint
p1,p2 = input_model_from_db(data_fp_true[:size_true*size_percentage])

  0%|          | 0/180825 [00:00<?, ?it/s]

In [7]:
#On récupère les produits et les réactions correspondantes sous forme de fingerprint pour la base de données fausses
#Dans le but de ne pas induire un biais, on garde la même proportion de données vraies et fausses
#Nous avons essayé de shuffle les données avant et d'entraîner notre modèle mais cela n'a pas donné de 
#différences notables 
#Nous avons donc gardé la version la plus simple
p3,p4 = input_model_from_db(data_fp_false[:size_false*size_percentage])

  0%|          | 0/25076 [00:00<?, ?it/s]

In [8]:
#Array des Fingerprints des Produits liés aux Vraies Reactions et Fausses Reactions
prod_fp = np.concatenate((p1, p3))prod_fp = np.concatenate((p1, p3))

In [9]:
#Array des Fingerprints des Réactions liés aux Vraies Reactions et Fausses Reactions
rxn_fp = np.concatenate((p2,p4))

In [10]:
#Labels : 1 pour les vraies réactions, 0 pour les fausses
labels = np.array([1]*p1.shape[0]+[0]*p3.shape[0])

In [11]:
#### PARAMETRES : proportion d'entraînement
train_prop = 0.9
test_prop  = 0.1

In [12]:
#On divise les données de façon aléatoire : on utilise le random_state 42 pour avoir des résultats reproductibles
prd_fp_train, prd_fp_test, rxn_fp_train, rxn_fp_test, labels_train, labels_test = train_test_split(prod_fp, rxn_fp, labels, train_size = train_prop, random_state=42)

# Entrainement du Modele

In [14]:
#On crée notre modèle : on choisit 100 estimateurs ; avec 1000, après un jour à tourner l'algorithme 
#n'avait toujours pas fini 
clf = RandomForestClassifier(n_estimators = 100)

In [15]:
#On écrit la data en input sous la forme désirée
data_input = [np.concatenate((rxn_fp_train[i], prd_fp_train[i])) for i in range(rxn_fp_train.shape[0])]

In [22]:
#On entraîne le modèle
clf.fit(data_input[:50000], labels_train[:50000])

RandomForestClassifier()

In [23]:
#On sauve le modèle 
filename = 'Random_Forest_model.sav'
pickle.dump(clf, open(filename, 'wb'))

# Premiers résultats

In [31]:
#On récupère la data de test
data_test = [np.concatenate((rxn_fp_test[i], prd_fp_test[i])) for i in range(rxn_fp_test.shape[0])]

In [32]:
#On prédit
test_predictions = clf.predict(data_test)

In [34]:
#On calcule précision, recall et fscore
precision, recall, fscore, support = score(labels_test, test_predictions, pos_label=1, average='binary')

In [35]:
#On affiche
precision, recall, fscore

(0.8971525288729589, 0.9976473193279637, 0.9447349452853678)